In [1]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
import scipy, numpy

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
# Caminhos para as pastas de dados
train_data_dir = './data'
batch_size = 2 # ***
image_size = (150, 150)
num_epochs = 10
num_classes = 2

In [5]:
# Crie geradores de dados de treinamento
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255, # Normalização das imagens
    validation_split=0.20,  # Define a divisão de validação
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)    

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 16 images belonging to 2 classes.


In [7]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 4 images belonging to 2 classes.


In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(image_size[0], image_size[1], 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Camada de saída com 1 unidade para classificação binária

In [9]:
# Compile o modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
# Treine o modelo
model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator)

# Salve o modelo treinados
model.save('./models/modelo_arvore-flor.h5')

print('Modelo salvo com sucesso!')

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.